In [4]:
#%serialconnect to --port="/dev/tty.usbserial-02U1W54L" --baud=115200
%serialconnect to --port="/dev/tty.usbserial-0001" --baud=115200

Connecting to --port=/dev/tty.usbserial-0001 --baud=115200 
Ready.


In [15]:
import gc
from micropython import alloc_emergency_exception_buf

# allocate exception buffer for ISRs
alloc_emergency_exception_buf(100)

# enable and configure garbage collection
gc.enable()
gc.collect()
gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())

from machine import Pin
import time

p26 = Pin(26, Pin.OUT)  #green
p13 = Pin(13, Pin.OUT)  #red
buttonA = Pin(32, Pin.IN)
buttonB = Pin(34, Pin.IN)
p26.on()
p13.on()

counter = 0
while counter !=10:
    if (counter % 2 == 0):
        p26.on()
        p13.off()
    else:
        p13.on()
        p26.off()
    time.sleep(1)
    counter+=1

p26.off()
p13.off()

time.sleep(5)

from lib.utils import connect_wifi, load_env_vars

env_vars = load_env_vars("lib/.env")
# connect WiFI
ssid = env_vars.get("WIFI_SSID")
password = env_vars.get("WIFI_PASSWORD")
connect_wifi(ssid, password)

# # import webrepl
# # webrepl.start()

p26.on()
p13.on()

time.sleep(5)

from ulab import numpy as np
import utime as time
from lib.runner import Runner

def collectData(decode_period_s):
    
    global runner
    
    time.sleep(decode_period_s)
    data = runner.output_buffer
    gc.collect()
    return np.array(data)

def getData(Nt, decode_period_s):

    global runner
    global p26

    runner.run()
    
    trials = []
    time.sleep(5)
    count=0
    
    toggle = True

    if Nt <=1:
        return collectData(decode_period_s)
    for i in range(Nt):
        if toggle:
            p26.on()
            toggle = False
        else:
            p26.off()
            toggle = True

        trials.append(collectData(decode_period_s).flatten())
        
    runner.stop()

    gc.collect()
    p26.off()
    return np.array(trials)

decode_period_s = 4 # decode every x seconds
Nc = 1
Ns = 128
Nt = 3
stim_freqs = [7, 10, 12]

runner = Runner('CCA', buffer_size=Ns)
runner.setup() 

p26.off()
p13.off()

calibration_data = {}

time.sleep(5)

p26.on()
p13.on()

# while True:
#     if not buttonB.value():
#         p26.on()
#         calibration_data[7] = getData(Nt, decode_period_s)
#         gc.collect()
#         p26.off()
#         break

calibration_data[7] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[10] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)
calibration_data[12] = getData(Nt, decode_period_s)
print(gc.mem_free())
gc.collect()
print(gc.mem_free())
p26.off()
p13.on()
time.sleep(5)

p26.on()
p13.on()

time.sleep(5)

p26.on()
p13.off()

time.sleep(5)

decode = Runner('MsetCCA', buffer_size=Ns) # initialise a base runner
decode.setup() # setup peripherals and memory buffers
decode.calibrate(calibration_data)

decode.run()

for i in range(5):
    p13.on()
    time.sleep(5)
    print(decode.decoder.classify(np.array(decode.output_buffer)),print(decode.decoded_output))
    p13.off()

decode.stop()

...network config: ('192.168.0.76', '255.255.255.0', '192.168.0.1', '194.168.4.100')
.ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
....53872
60608
.....53312
60048
....51712
58448
...ADC initialised
SPI initialised
DigiPot set to 100 = gain of 10.62498
.{12: 0.29706, 10: 0.55054, 7: 0.07951}
{12: 0.2970607, 10: 0.5505426, 7: 0.07950612} None
.{12: 0.29694, 10: 0.19762, 7: 0.28244}
{12: 0.2969436, 10: 0.1976208, 7: 0.2824449} None
.{12: 0.01402, 10: 0.42752, 7: 0.36371}
{12: 0.01402025, 10: 0.4275179, 7: 0.3637127} None
.{12: 0.02404, 10: 0.37772, 7: 0.20022}
{12: 0.02404299, 10: 0.3777189, 7: 0.2002219} None
.{12: 0.05775, 10: 0.37567, 7: 0.28524}
{12: 0.1084236, 10: 0.2572649, 7: 0.4159674} None
